# Intro

This notebook tries to explore the problem of sentence segmentation or sentence boundary disambiguation.  It also will correct the EEUU problems found in the text

# Sentence boundary disambiguation

In [1]:
from nltk.tokenize import sent_tokenize

In [4]:
text = "this's a sent tokenize test. this is sent two. is this sent three? sent 4 is cool! Now it's your turn."
sent_tokenize_list = sent_tokenize(text)
print len(sent_tokenize_list)

sent_tokenize_list

5


["this's a sent tokenize test.",
 'this is sent two.',
 'is this sent three?',
 'sent 4 is cool!',
 "Now it's your turn."]

In [12]:
text = u"Corea del Norte lanzó un misil balístico este viernes, según le confirmó a CNN el vocero del Pentágono, \
el capital Jeff Davis. EE.UU. detectó el lanzamiento a las 10:45 EDT.\n\
El jefe de gabinete de Japón, Yoshihide Suga, dio una rueda de prensa este sábado (hora local) y dijo que el misil \
que lanzó Corea del Norte probablemente voló por 45 minutos.\n\
Suga les dijo a los reporteros que no hay daño alguno a navíos o aeronaves.\
 El funcionario agregó que se cree que el misil cayó en una zona económica exclusiva de Japón."
sent_tokenize_list = sent_tokenize(text)
print len(sent_tokenize_list)

sent_tokenize_list

6


[u'Corea del Norte lanz\xf3 un misil bal\xedstico este viernes, seg\xfan le confirm\xf3 a CNN el vocero del Pent\xe1gono, el capital Jeff Davis.',
 u'EE.UU.',
 u'detect\xf3 el lanzamiento a las 10:45 EDT.',
 u'El jefe de gabinete de Jap\xf3n, Yoshihide Suga, dio una rueda de prensa este s\xe1bado (hora local) y dijo que el misil que lanz\xf3 Corea del Norte probablemente vol\xf3 por 45 minutos.',
 u'Suga les dijo a los reporteros que no hay da\xf1o alguno a nav\xedos o aeronaves.',
 u'El funcionario agreg\xf3 que se cree que el misil cay\xf3 en una zona econ\xf3mica exclusiva de Jap\xf3n.']

In [14]:
import pandas as pd 
import numpy as np
import bs4 #beautiful soup

In [16]:
base_df = pd.read_csv("../files/articles.csv") 

In [17]:
base_df.head()

,id,date,title,content,tags
0,post-80480,"28 Dic, 2009 - 7:27 pm",Auditoría revela irregularidades en el Parlacen,"<div class=""article-post-content""><p><strong><...",None
1,post-80485,"28 Dic, 2009 - 7:32 pm",Suspendidas las citas en Hospital Escuela,"<div class=""article-post-content""><p><strong><...",None
2,post-80555,"28 Dic, 2009 - 8:25 pm",Mariscos contaminados alarman a los “porteños”,"<div class=""article-post-content""><p><strong><...",None
3,post-80559,"28 Dic, 2009 - 8:26 pm",Citan a 11 personas por vender pólvora,"<div class=""article-post-content""><p><strong><...",None
4,post-80578,"29 Dic, 2009 - 1:00 am",Con compra de granos se paliaría hambruna en e...,"<div class=""article-post-content""><p><strong><...",None


In [34]:
article = base_df.loc[2298]
print article.title
raw_text = article["content"]
soup = bs4.BeautifulSoup(raw_text, 'html.parser')
text = soup.get_text()

print text
sent_tokenize_list = sent_tokenize(text)
print len(sent_tokenize_list)

for sent in sent_tokenize_list:
    print
    print sent

Un muerto y un herido en un tiroteo ante la Agencia de Seguridad de EE.UU.
Washington, (EFE).- Al menos una persona falleció hoy y otra resultó herida en un tiroteo ocurrido en uno de los accesos a la base militar de Fort Meade (Maryland), sede de la Agencia de Seguridad Nacional (NSA), según informó el canal local NBC 4 en EE.UU.
Imágenes de televisión muestran dos vehículos dañados por una posible colisión, uno de ellos claramente de la policía.
El tiroteo se produjo después de que dos personas intentaron acceder en un todoterreno a la base sin autorización.
La cadena local NBC 4 aseguró que los ocupantes del automóvil eran dos hombres, y en el vehículo que conducían se encontraron drogas y un arma de fuego.
Las imágenes de televisión también mostraron a un policía herido, así como una persona tapada con una sábana que yace en el suelo, aparentemente la víctima mortal del tiroteo.
Aún se desconocen las razones del incidente y la Policía ha declinado dar detalles, porque el altercado 

In [30]:
article = base_df.loc[2298]
raw_text = article["content"]
soup = bs4.BeautifulSoup(raw_text, 'html.parser')
text = soup.get_text().replace("\n","")
sentences = text.split(".")
print len(sentences)

for sent in sentences:
    print
    print sent

12

Washington, (EFE)

- Al menos una persona falleció hoy y otra resultó herida en un tiroteo ocurrido en uno de los accesos a la base militar de Fort Meade (Maryland), sede de la Agencia de Seguridad Nacional (NSA), según informó el canal local NBC 4 en EE

UU

Imágenes de televisión muestran dos vehículos dañados por una posible colisión, uno de ellos claramente de la policía

El tiroteo se produjo después de que dos personas intentaron acceder en un todoterreno a la base sin autorización

La cadena local NBC 4 aseguró que los ocupantes del automóvil eran dos hombres, y en el vehículo que conducían se encontraron drogas y un arma de fuego

Las imágenes de televisión también mostraron a un policía herido, así como una persona tapada con una sábana que yace en el suelo, aparentemente la víctima mortal del tiroteo

Aún se desconocen las razones del incidente y la Policía ha declinado dar detalles, porque el altercado se produjo en propiedad de la NSA, agencia estadounidense de espionaj

# CORRECT ALL "EE.UU." instances problems

In [39]:
import sqlite3
import json
conn = sqlite3.connect('../articles.db')
cursor = conn.execute('''
    SELECT ID from ARTICLE
    WHERE ID = 1 
    '''
)
articles = [{"id": row[0], "title": "Article "+str(row[0]) } for row in cursor]
articles

[{'id': 1, 'title': 'Article 1'}]

In [76]:
def getArticleContent(art_id, conn):
    values = [art_id]
    cursor = conn.execute('''
    SELECT SENTENCES  from ARTICLE
    WHERE ID = ?
    ''',values
    )
    
    for row in cursor:
        article = {
            "sentences": json.loads(row[0]),
        }
    return article

def findEEUU(art_id,conn):
    article = getArticleContent(art_id, conn)
    
    for sent in article["sentences"]:
        for word in sent:
            if word[0] == "EE" or word[0] == "UU":
                return True
    
    return False
    
eeuu_articles = []
for i in range(2500):
    if findEEUU(i,conn):
        eeuu_articles.append(i)

len(eeuu_articles)

0

In [77]:
eeuu_articles

[]

In [69]:
def updateArticle(art_id,sentences,conn):
    sentences = json.dumps(sentences)
    values = ( sentences, art_id )

    conn.execute('''
    UPDATE ARTICLE set  SENTENCES = ? where ID=?
    ''', values)
    conn.commit()
    print "Updated Article"

In [75]:
def correct(art_id, conn):
    print "- Get Article"
    article = getArticleContent(art_id, conn)
    
    print "- Correct Article"
    last = ""
    last_pos = -1
    last_sent = -1
    for sent_id, sent in enumerate(article["sentences"]):
        for pos, word in enumerate(sent):
            if word[0] == "UU" and last == "EE":
                article["sentences"][last_sent][last_pos][0] += ".UU."
                del article["sentences"][sent_id][pos]
            last = word[0]
            last_pos = pos
            last_sent = sent_id

#     for sent_id, sent in enumerate(article["sentences"]): 
#         print 
#         print " ".join([word[0] for word in sent])

    print "- Update Article"
    updateArticle(art_id,article["sentences"],conn)
        
for art_id in  eeuu_articles:    
    print "-----"
    print "article %i" %art_id
    correct(art_id, conn)             

-----
article 17
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 72
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 208
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 306
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 307
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 316
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 317
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 325
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 341
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 350
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 351
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 352
- Get Article
- Correct Article
- Update Article
Updated Article
-----


# Correct Titles

In [105]:
def getArticleTitle(art_id, conn):
    values = [art_id]
    cursor = conn.execute('''
    SELECT TITLE  from ARTICLE
    WHERE ID = ?
    ''',values
    )
    
    for row in cursor:
        article = {
            "title": json.loads(row[0]),
        }
    return article

def findEEUU_TITLE(art_id,conn):
    article = getArticleTitle(art_id, conn)
    
    for pos, word in enumerate(article["title"]):
        if word[0] == "EE" and article["title"][pos+1][0] == "." and article["title"][pos+2][0] == "UU":
            return True
    return False
    
eeuu_articles = []
for i in range(2500):
    if findEEUU_TITLE(i,conn):
        eeuu_articles.append(i)

len(eeuu_articles)

7

In [106]:
eeuu_articles

[605, 992, 1637, 2056, 2135, 2298, 2480]

In [99]:
def updateArticleTitle(art_id,title,conn):
    title = json.dumps(title)
    values = ( title, art_id )

    conn.execute('''
    UPDATE ARTICLE set  TITLE = ? where ID=?
    ''', values)
    conn.commit()
    print "Updated Article"

In [107]:
def correct_title(art_id, conn):
    print "- Get Article"
    article = getArticleTitle(art_id, conn)
    
    print "- Correct Article"
    last = ""

    for pos, word in enumerate(article["title"]):
        if word[0] == "UU" and last == ".":
            article["title"][pos-2][0] += ".UU."
            if article["title"][pos+1][0] == ".":
                del article["title"][pos+1]
            del article["title"][pos]
            del article["title"][pos-1]
        last = word[0]

#     print " ".join([word[0] for word in article["title"]])
    print "- Update Article"
    updateArticleTitle(art_id,article["title"],conn)
        
for art_id in  eeuu_articles:    
    print "-----"
    print "article %i" %art_id
    correct_title(art_id, conn)   



-----
article 605
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 992
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 1637
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 2056
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 2135
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 2298
- Get Article
- Correct Article
- Update Article
Updated Article
-----
article 2480
- Get Article
- Correct Article
- Update Article
Updated Article


In [103]:
conn2 = sqlite3.connect('../articles_backup.db')

article = getArticleContent(132, conn2)
updateArticle(132,article["sentences"],conn)

Updated Article
